In [ ]:
import numpy as np
import xarray as xr
import rasterio
from rasterio.transform import from_origin

In [ ]:
def netcdf_layer_to_geotiff(input_path, layer_index, output_path,
                             var_name='combined_layers',
                             lat_name='lat', lon_name='lon',
                             crs='EPSG:4326'):
    """
    Reads a specified layer from a NetCDF variable and writes it to a GeoTIFF.

    Parameters:
    - input_path: Path to the input NetCDF file.
    - layer_index: Integer index of the layer to extract.
    - output_path: Path where the output GeoTIFF will be saved.
    - var_name: Name of the variable in the NetCDF containing layers.
    - lat_name: Name of the latitude coordinate (1D array).
    - lon_name: Name of the longitude coordinate (1D array).
    - crs: Coordinate reference system for the output GeoTIFF.
    """
    # Open the NetCDF dataset
    ds = xr.open_dataset(input_path)

    # Extract the 2D layer
    try:
        data_arr = ds[var_name].isel(layer=layer_index)
    except KeyError:
        raise KeyError(f"Variable '{var_name}' not found in dataset.")
    except IndexError:
        raise IndexError(f"Layer index {layer_index} out of bounds.")

    # Get coordinate arrays
    lats = ds[lat_name].values
    lons = ds[lon_name].values
    if lats.ndim != 1 or lons.ndim != 1:
        raise ValueError("Latitude and longitude must be 1D arrays.")

    data = data_arr.values

    # Compute resolution
    lat_res = abs(lats[1] - lats[0])
    lon_res = abs(lons[1] - lons[0])

    # Define affine transform
    transform = from_origin(
        west=lons.min() - lon_res/2,
        north=lats.max() + lat_res/2,
        xsize=lon_res,
        ysize=lat_res
    )

    # Write GeoTIFF
    with rasterio.open(
        output_path, 'w',
        driver='GTiff',
        height=data.shape[0],
        width=data.shape[1],
        count=1,
        dtype=data.dtype,
        crs=crs,
        transform=transform
    ) as dst:
        dst.write(data, 1)

    print(f"GeoTIFF saved to: {output_path}")


In [ ]:
### run
input_nc   = r"C:\Users\TyHow\Desktop\dh.nc"
output_tif = r"C:\Users\TyHow\Desktop\dh.tif"
layer_idx  = 0  


netcdf_layer_to_geotiff(input_nc, layer_idx, output_tif, lat_name='y', lon_name='x')

GeoTIFF saved to: C:\Users\TyHow\Desktop\dh.tif
